In [8]:
import subprocess
import os

In [ ]:
# Задаем список наборов аргументов для 20 запусков
arguments_list = [
    [0, 0, 0, 100],
    [5, 6, 7, 8],
    # Добавьте остальные наборы аргументов по аналогии
]

In [16]:
def generate_lists():
    lists = []
    p_max = 11
    d_max = 11
    q_max = 11
    max_iteration_num = int(p_max * q_max * d_max / (os.cpu_count()))
    iteration_num = 0
    for i in range(p_max):
        for j in range(d_max):
            for k in range(q_max):
                iteration_num += 1
                if iteration_num % max_iteration_num  == 0:
                    iteration_num = 0
                    lists.append([i, j, k, max_iteration_num])
    lists[len(lists) - 1][3] += (p_max * q_max * d_max - max_iteration_num * 20)
    return lists

num_threads_os = os.cpu_count()
print(num_threads_os)
result_lists = generate_lists()
print(result_lists)

20
[[0, 5, 10, 66], [1, 0, 10, 66], [1, 6, 10, 66], [2, 1, 10, 66], [2, 7, 10, 66], [3, 2, 10, 66], [3, 8, 10, 66], [4, 3, 10, 66], [4, 9, 10, 66], [5, 4, 10, 66], [5, 10, 10, 66], [6, 5, 10, 66], [7, 0, 10, 66], [7, 6, 10, 66], [8, 1, 10, 66], [8, 7, 10, 66], [9, 2, 10, 66], [9, 8, 10, 66], [10, 3, 10, 66], [10, 9, 10, 77]]


In [ ]:
# Запускаем 20 экземпляров скрипта с разными аргументами
for i, arguments in enumerate(arguments_list, start=1):
    command = ["python", "ARIMATest.py"] + [str(arg) for arg in arguments]
    try:
        # Запускаем подпроцесс
        subprocess.run(command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Ошибка при запуске скрипта {i}: {e}")

In [18]:
import sys
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import warnings

# Функция для построения модели ARIMA и получения прогноза
def build_arima_model_and_forecast(data, p, d, q, window_size = 200, forecast_days = 90):
    forecasts = []
    for i in range(len(data) - window_size - forecast_days + 1):
        try:
            window = data[i:i + window_size]
            actual_values = data['Close'][i + window_size:i + window_size + forecast_days]
            # Создаем и обучаем модель ARIMA
            model = ARIMA(window['Close'], order = (p, d, q))
            results = model.fit()

            # Делаем прогноз на 90 дней вперед
            forecast = results.get_forecast(steps=forecast_days)
            forecast_mean = forecast.predicted_mean.values

            # Рассчитываем относительное отклонение в процентах (Percentage Error)
            percentage_error = ((forecast_mean - actual_values) / actual_values) * 100

            # Добавляем результаты в список
            result = {
                'p': p,
                'd': d,
                'q': q,
                'begin_date': window['Date'].iloc[0],
                'end_date': window['Date'].iloc[-1] + pd.Timedelta(days=forecast_days - 1),
                'window_size': window_size,
                'forecast_days': forecast_days,
                'forecast_precision': np.mean(np.abs(percentage_error))
            }
            forecasts.append(result)
        except:
            continue

    return forecasts

def load_and_transform_data():
    #load data
    data = pd.read_csv("..\..\..\Data\Day\dje_MinuteToDay.csv")
    #transform data
    data = data.replace(",", "", regex=True)
    data.Date = pd.to_datetime(data.Date)
    data = data.sort_values(by=["Date"])
    data.set_index('Date')
    data.Close = data.Close.astype(float)
    data.High = data.High.astype(float)
    data.Low = data.Low.astype(float)
    data.Open = data.Open.astype(float)
    data["Vol"] = data["Vol"].astype(float)
    # раз
    data.index = data.index[::-1]
    #
    special_data = data[(data['Date'] > '2010-01-01') & (data['Date'] < '2014-01-01')]
    special_data.index = data.index[:len(special_data)]
    #
    special_data = special_data.drop(columns=['Open', 'High', 'Low', 'Vol', 'Ticker', 'Per', 'Date.1'])
    return special_data

def save_result(results_df, p_begin, d_begin, q_begin, iteration_num):
    results_df.to_csv(f'output{p_begin}_{d_begin}_{q_begin}_{iteration_num}.csv', encoding='utf8')

def calculate_ARIMA(p_begin, d_begin, q_begin, iteration_num):
    # Указываем наши промежутки
    p_values = range(p_begin, 11)
    d_values = range(d_begin, 11)
    q_values = range(q_begin, 11)
    # Получаем данные
    special_data = load_and_transform_data()
    # Создаем пустой DataFrame для сохранения результатов
    columns = ['p', 'd', 'q', 'begin_date', 'end_date', 'window_size', 'forecast_days', 'forecast_precision']
    results_df = pd.DataFrame(columns = columns)
    counter = 0
    print(special_data.head(1))
    # Проходим по всем комбинациям p, d, q и строим прогнозы
    for p in p_values:
        for d in d_values:
            for q in q_values:
                if counter >= iteration_num:
                    print(results_df.head(2))
                    save_result(results_df, p_begin, d_begin, q_begin, iteration_num)
                    return
                counter+= 1
                print(f'Calculating:({p}, {d}, {q}). Counter: {counter}. Iter_num: {iteration_num}. Equals {counter >= iteration_num}')
                # Строим прогноз для текущих значений p, d, q
                forecasts = build_arima_model_and_forecast(special_data, p, d, q)
                # Добавляем результаты в DataFrame
                results_df = pd.concat([results_df, pd.DataFrame(forecasts)], ignore_index = True)#results_df.append(forecasts, ignore_index=True)


def main():
    warnings.filterwarnings('ignore')
    print("Я стартанул!")
    # Проверим, что передано четыре аргумента
    if len(sys.argv) != 5:
        print("Использование: python myscript.py <p> <d> <q> <iter_num>")
        return

    try:
        # Преобразуем аргументы командной строки в целые числа
        num1 = int(sys.argv[1])
        num2 = int(sys.argv[2])
        num3 = int(sys.argv[3])
        num4 = int(sys.argv[4])

        calculate_ARIMA(num1, num2, num3, num4)
    except ValueError as ve:
        print(f"Ошибка: {ve}")
    except Exception as e:
        print(f"Произошла ошибка: {e}")

if __name__ == "__main__":
    main()

Я стартанул!
Использование: python myscript.py <p> <d> <q> <iter_num>


In [19]:
calculate_ARIMA(0, 7, 5, 1)

           Date     Close
1350 2010-01-04  10583.13
Calculating:(0, 7, 5). Counter: 1. Iter_num: 1. Equals True
1350
